# Imports

In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.metrics import mean_squared_error

# Load data

In [2]:
data_directory = '../input/gem-predictive-maintenance-2022'
chosen_dataset = 'FD003'

df_train = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_train.csv'), index_col=0) 
df_test = pd.read_csv(os.path.join(data_directory, chosen_dataset + '_df_test.csv'), index_col=0) 

# Look at the dataset

In [3]:
df_train

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_18,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,2388,100.0,39.11,23.3537,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,2388,100.0,38.99,23.4491,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,2388,100.0,38.85,23.3669,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,2388,100.0,38.96,23.2951,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,2388,100.0,39.14,23.4583,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,2388,100.0,38.44,22.9631,NaN,NaN,NaN,NaN,NaN,NaN
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,2388,100.0,38.50,22.9746,NaN,NaN,NaN,NaN,NaN,NaN
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,2388,100.0,38.39,23.0682,NaN,NaN,NaN,NaN,NaN,NaN
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,2388,100.0,38.31,23.0753,NaN,NaN,NaN,NaN,NaN,NaN


The dataset contains 32 colums.

The columns engine_no and time_in_cycles are metadata columns. It enable to identify an engine at a certain point in time.

All the other columns (op_settings_1..2 and sensors_1..27) refer to data columns. It stores sensor data for a given engine at a given point in time.

# Add the RUL

In the current dataset the RUL is not part of the column.

In the data descrition, it is mentionned that in the training data, the engine runs until failure.

Given the time in cycles, it is easy to retreive the RUL.



In [4]:
def add_rul(g):
    g['RUL'] = max(g['time_in_cycles']) - g['time_in_cycles']
    return g

df_train = df_train.groupby('engine_no').apply(add_rul)

In [5]:
df_train

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_19,sensor_20,sensor_21,sensor_22,sensor_23,sensor_24,sensor_25,sensor_26,sensor_27,RUL
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,100.0,39.11,23.3537,NaN,NaN,NaN,NaN,NaN,NaN,258
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,100.0,38.99,23.4491,NaN,NaN,NaN,NaN,NaN,NaN,257
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,100.0,38.85,23.3669,NaN,NaN,NaN,NaN,NaN,NaN,256
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,100.0,38.96,23.2951,NaN,NaN,NaN,NaN,NaN,NaN,255
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,100.0,39.14,23.4583,NaN,NaN,NaN,NaN,NaN,NaN,254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,100.0,38.44,22.9631,NaN,NaN,NaN,NaN,NaN,NaN,4
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,100.0,38.50,22.9746,NaN,NaN,NaN,NaN,NaN,NaN,3
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,100.0,38.39,23.0682,NaN,NaN,NaN,NaN,NaN,NaN,2
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,100.0,38.31,23.0753,NaN,NaN,NaN,NaN,NaN,NaN,1


# Explanatory data analysis (EDA)

In [6]:
dataset_description = df_train.describe()

In [7]:
#axes = dataset_description.T.plot.bar(subplots=True, figsize=(15,10))


In [8]:
nan_column = df_train.columns[df_train.isna().any()].tolist()
const_columns = [c for c in df_train.columns if len(df_train[c].drop_duplicates()) <= 2]
print('Columns with all nan: \n' + str(nan_column) + '\n')
print('Columns with all const values: \n' + str(const_columns) + '\n')

df_train = df_train[[c for c in df_train.columns if c not in nan_column + const_columns]]

Columns with all nan: 
['sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27']

Columns with all const values: 
['op_setting_3', 'sensor_1', 'sensor_5', 'sensor_16', 'sensor_18', 'sensor_19', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25', 'sensor_26', 'sensor_27']



In [9]:
#df_corr = df_train.corr(method='pearson')
#fig, ax = plt.subplots(figsize=(15,15))
#axes = sns.heatmap(df_corr, linewidths=.2, annot= True )

graph = sns.PairGrid(data=df_train, x_vars="RUL", y_vars=df_train.columns, hue="engine_no", height=4, aspect=6,)
graph = graph.map(plt.plot, alpha=0.5)
graph = graph.set(xlim=(df_train['RUL'].max(),df_train['RUL'].min()))

# Make a first model

In [10]:
RATIO = .2
number_of_engine_no = df_train['engine_no'].max()
number_engine_val = int(number_of_engine_no * RATIO)
engine_no_val = random.sample(range(1, number_of_engine_no + 1), number_engine_val)
engine_no_train = [nb for nb in range(1, number_of_engine_no + 1) if nb not in engine_no_val]

In [11]:
selected_features = [ 
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_7', 'sensor_8',
       'sensor_9', 'sensor_11', 'sensor_12', 'sensor_13',
       'sensor_14',  'sensor_17']

In [12]:
data_train = df_train[df_train['engine_no'].isin(engine_no_train)]
data_val = df_train[df_train['engine_no'].isin(engine_no_val)]

X_train, y_train = data_train[selected_features], data_train['RUL'] 
X_val, y_val = data_val[selected_features], data_val['RUL']
X_test = df_test[selected_features]

X_all, y_all = df_train[selected_features], df_train['RUL']
print (data_train)

       engine_no  time_in_cycles  op_setting_1  op_setting_2  sensor_2  \
0              1               1       -0.0005        0.0004    642.36   
1              1               2        0.0008       -0.0003    642.50   
2              1               3       -0.0014       -0.0002    642.18   
3              1               4       -0.0020        0.0001    642.92   
4              1               5        0.0016        0.0000    641.68   
...          ...             ...           ...           ...       ...   
24715        100             148       -0.0016       -0.0003    643.78   
24716        100             149        0.0034       -0.0003    643.29   
24717        100             150       -0.0016        0.0004    643.84   
24718        100             151       -0.0023        0.0004    643.94   
24719        100             152        0.0000        0.0003    643.64   

       sensor_3  sensor_4  sensor_6  sensor_7  sensor_8  ...  sensor_10  \
0       1583.23   1396.84     21.61 

In [13]:
from xgboost import XGBRegressor 

xgb = XGBRegressor(learning_rate = 0.3, max_depth = 5, n_estimators = 180, subsample = 0.7, colsample_bylevel = 0.7, colsample_bytree = 0.7, min_child_weight = 4, reg_alpha = 10, reg_lambda = 10)
xgb.fit(X_train, y_train)
preds = xgb.predict(X_val)

In [14]:
print("Score on train data : " + str(xgb.score(X_train, y_train)))
print("Score on validation data : " + str(xgb.score(X_val, y_val)))
rmse = np.sqrt(mean_squared_error(y_val, preds))
print(rmse)

Score on train data : 0.7709676426447601
Score on validation data : 0.6505873334832317
62.954158065781805


In [15]:
df_test['RUL'] = xgb.predict(X_test)
df_sub = df_test.groupby('engine_no').agg({'RUL': 'last'}).reset_index()
df_sub[['engine_no','RUL']].to_csv('submission.csv', index=False)

In [16]:
df_sub

,engine_no,RUL
0,1,59.840744
1,2,96.637398
2,3,37.055202
3,4,124.095154
4,5,190.569122
...,...,...
95,96,53.532288
96,97,228.373764
97,98,15.202318
98,99,8.095011


# You work for the next lesson:

By groups of 3 :
- Try to have the best model on FD003
- Prepare a quick presentation (5 min) explaining your data analysis and you model.
- Explain how you would answer to the following issue :
«I need to predict failure on my engine. Considering that each engine does 8 cycles a day on average. If my engine fails, it costs 4m to repair, and 500k of operational impact. If I repair the engine before failure it costs me 1m and depending on removal planning, costs are different. If I plan my maintenance within 5 days it costs me 400k of operational impact. From 5 to 10 days 200k, and above 10 days 100k. Also I know that if I remove an engine before it fails I lose some potential (~3,5k / cycle);»